In [112]:
from __future__ import annotations
import numpy as np
import math
from itertools import permutations

class Permutation:
    @classmethod
    def get_permutations(cls, n, per=False): # get list of permutations for given 'n'
        per = permutations(range(1, n+1), n)
        if per:
            return [cls(n, p) for p in per]
        else:
            return [ p for p in per]
    @classmethod
    def reverse_permutation(cls, n):
        return cls(n, range(n, 0, -1)) 

    def __init__(self, n, plist):
        if len(plist) != n:
            raise ValueError(f"{n} must be same with len(plist) = {len(plist)}")

        if sum(plist) != int(n* (n+1)/2):
            raise ValueError(f"plist does not satisfy permutation proeprty.\n All [0, n-1] values must be in plist. \n {plist}")

        self.n = n
        self.plist = plist
    def __getitem__(self, key):
        if type(key) != int:
            raise ValueError(f"key must be an integer type element: {key}")
        if key < 1 or key > self.n:
            raise IndexError(f"{key} must be in [1, {self.n}] range.")
        
        return self.plist[key-1]
    def __mul__(self, other: Permutation) -> Permutation:
        if self.n != other.n:
            raise ValueError(f"{self.n} and {other.n} are not same.")
        
        rlist = [other[x] for x in self.plist]
        return Permutation(self.n, rlist)
    
    def inverse(self)-> Permutation:
        ilist= [self.plist.index(x)+1 for x in range(1, self.n+1)]
        return Permutation(self.n, ilist)
    
    def notation_cauchy(self):
        return f"{list(range(1, self.n+1))}\n{self.plist}"

    
    def __canocial_order(self, cyclist):
        c_clist_1 = []
        c_clist_2 = []
        for li in cyclist:
            n = len(li)
            nmax = li.index(max(li))
            c_clist_1.append([li[x] for x in range(nmax-n, nmax)])
        maxindex = [x[0] for x in c_clist_1]
        s_index = sorted(maxindex)

        for i in s_index:
            c_clist_2.append(c_clist_1[maxindex.index(i)])
        
        return c_clist_2

    def notation_cyclic(self, canonical=False, string= False, sep=''):
        cycliclist = []
        index = list(range(1, self.n+1))
        i =1

        while True:
            ilist = []
            ilist.append(i)
            index.remove(i)
            
            sig_i = self[i]
            while sig_i in index:
                ilist.append(sig_i)
                index.remove(sig_i)
                sig_i = self[sig_i]
            
            cycliclist.append(ilist)

            if len(index) > 0:
                i = min(index)
            else: 
                break
        if canonical:
            cycliclist = self.__canocial_order(cycliclist)
        
        if string:
            cycstr = ''
            for li in cycliclist:
                cycstr = cycstr+ '('+f'{sep}'.join(map(str,li))+')'
            return cycstr
        else:
            return cycliclist

per_fold ={
    4: [
        [4,1],
        [2,3]
        ],
    8: [
        [8,1,5,4],
        [2,7,3,9]
        ],
    16: [
        [16,1,4,13,9,8,5,12],
        [10,7,6,11,15,2,3,14]
        ]
}

def sig_layout(n):
    if type(n) != int or n<4 or n%4 !=0:
        raise ValueError(f"n:{n} must be a positive integer that multiple of 4.")
    
    i = int(math.log(n/4 ,2))
    if i%2 :
        k = kp = int((i+1)/2)
    else:
        k = int(i/2)
        kp = k+1
    return (int(2**k), int(2**kp))

def __sig_matrix_update(n, matrix):
    n_1 = np.flip(matrix.T, axis=0)
    len_n = len(n_1[0])
    l = int(len_n/2)

    rows =[]
    for row in n_1:
        r_split = np.split(row,l)
        row_appended = []

        for tu in r_split:
            tem = np.array([n-tu[0] +1,n-tu[1] +1])
            row_appended.append(np.insert(tem, 1, tu))    
        rows.append(np.concatenate(row_appended, axis=None))     
    return np.stack(rows)

def per_fold_n(n):
    if n % 4 !=0:
        raise ValueError("Fold sheets must be 4*2^k for k= 0, 1, 2, .... \n Current value is {n}")
    
    if n <32:
        return per_fold[n]
    else:
        n_iter = int(math.log(n/16,2))
        n_i = 16
        per_n_1 = [per_fold[16][0], per_fold[16][1]]

        #permutation to matrix
        layout_n_1 = sig_layout(16)
        front_matrix = np.array(per_n_1[0]).reshape(layout_n_1)
        back_matrix = np.array(per_n_1[1]).reshape(layout_n_1)
        for i in range(0, n_iter):
            n_i = 2*n_i
            front_matrix = __sig_matrix_update(n_i, front_matrix)
            back_matrix = __sig_matrix_update(n_i, back_matrix)
    print("Front")
    print(front_matrix)
    print("Back")
    print(back_matrix)
    
    per_fn = np.concatenate(front_matrix).tolist() 
    per_bn = np.concatenate(back_matrix).tolist()

    return [per_fn,per_bn]
                


In [113]:
per_fold_n(32)

Front
[[20 13 12 21]
 [29  4  5 28]
 [32  1  8 25]
 [17 16  9 24]]
Back
[[22 11 14 19]
 [27  6  3 30]
 [26  7  2 31]
 [23 10 15 18]]


[[20, 13, 12, 21, 29, 4, 5, 28, 32, 1, 8, 25, 17, 16, 9, 24],
 [22, 11, 14, 19, 27, 6, 3, 30, 26, 7, 2, 31, 23, 10, 15, 18]]

In [44]:
np16 = np16_f.reshape((2,4))
print(np16)

[[16  1  4 13]
 [ 9  8  5 12]]


In [64]:
n =32

In [65]:
f_n_1 = np.flip(np16.T, axis=0)
print(f_n_1)

[[13 12]
 [ 4  5]
 [ 1  8]
 [16  9]]


In [66]:
len_n = len(f_n_1[0])
l =int(len_n /2)

In [75]:
n_p1_f = []
for row in f_n_1:
    r_split = np.split(row, l)
    #print(r_split)
    row_appended = []
    for tu in r_split:
        tem = np.array([n-tu[0] +1,n-tu[1] +1])
        #print(np.insert(tem, 1, tu))
        row_appended.append(np.insert(tem, 1, tu))
    n_p1_f.append((np.concatenate(row_appended, axis=None)))


In [79]:
newarray = np.stack(n_p1_f)
print(newarray)

[[20 13 12 21]
 [29  4  5 28]
 [32  1  8 25]
 [17 16  9 24]]


In [80]:
list(newarray)

[array([20, 13, 12, 21]),
 array([29,  4,  5, 28]),
 array([32,  1,  8, 25]),
 array([17, 16,  9, 24])]

In [82]:
np.concatenate(newarray).tolist()

[20, 13, 12, 21, 29, 4, 5, 28, 32, 1, 8, 25, 17, 16, 9, 24]

In [3]:
import math

def sig_layout(n:int) -> tuple:
        if type(n) != int or n<4 or n%4 !=0:
            raise ValueError(f"n:{n} must be a positive integer that multiple of 4.")

        if n%3 ==0:
            i = math.log2(n) - math.log2(3) -1

            return(3, int(2**i))
            pass
        else:
            i = int(math.log2(n/4))
            if i%2 :
                k = kp = int((i+1)/2)
            else:
                k = int(i/2)
                kp = k+1
            return (int(2**k), int(2**kp))

In [6]:
sig_layout(24)

(3, 4)